<a href="https://colab.research.google.com/github/vanlicht/MachineLearningJourney/blob/master/tensorflow_practice_specialization/course_01_intro/week04/Exercise4_Question_Thomas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import os
import zipfile

DESIRED_ACCURACY = 0.999

!wget --no-check-certificate "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
-O '/tmp/happy-or-sad.zip'

--2020-08-12 04:01:59--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.133.128, 74.125.140.128, 108.177.15.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.133.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.02s   

2020-08-12 04:01:59 (127 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [2]:
zip_ref = zipfile.ZipFile('/tmp/happy-or-sad.zip', mode='r')
zip_ref.extractall(path='/tmp/h-or-s')
zip_ref.close()

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy') > DESIRED_ACCURACY):
      print("\nReached 99.9% accuracy so cancelling training!")
      self.model.stop_training=True

callbacks = myCallback()

In [3]:
train_happy_dir = os.listdir('/tmp/h-or-s/happy')
print(len(train_happy_dir))

40


In [4]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(filters=16, kernel_size=(3,3), activation=tf.nn.relu, input_shape=(150,150,3)),
  tf.keras.layers.MaxPool2D(pool_size=(2,2)),
  tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation=tf.nn.relu),
  tf.keras.layers.MaxPool2D(pool_size=(2,2)),
  tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation=tf.nn.relu),
  tf.keras.layers.MaxPool2D(pool_size=(2,2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(units=512, activation=tf.nn.relu),
  tf.keras.layers.Dense(units=1, activation=tf.nn.sigmoid)
  ])

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 18496)             0

In [6]:
from tensorflow.keras.optimizers import RMSprop
model.compile(optimizer=RMSprop(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1.0/255)
train_generator = train_datagen.flow_from_directory(
    directory='/tmp/h-or-s', 
    target_size=(150,150), 
    batch_size=10,
    class_mode='binary')

Found 80 images belonging to 2 classes.


In [8]:
history = model.fit(
    x=train_generator,
    steps_per_epoch=1,
    epochs=20, 
    verbose=1, 
    callbacks=[callbacks]
    )

Epoch 1/20
1/1 [==============================] - 0s 2ms/step - loss: 0.6814 - accuracy: 0.8000
Epoch 2/20
1/1 [==============================] - 0s 2ms/step - loss: 13.3428 - accuracy: 0.3000
Epoch 3/20
1/1 [==============================] - 0s 5ms/step - loss: 0.8481 - accuracy: 0.6000
Epoch 4/20
1/1 [==============================] - ETA: 0s - loss: 0.5701 - accuracy: 1.0000
Reached 99.9% accuracy so cancelling training!
1/1 [==============================] - 0s 5ms/step - loss: 0.5701 - accuracy: 1.0000


In [9]:
history.history['accuracy'][-1]

1.0